# CSCI 3155 Recitation 13

April 19, 2019

### Exercise
Given the following classes:
```scala
class A {
    def m1(): String = "m1 in A"
    def m2(): String = "m2 in A"
}

abstract class B extends A {
    def m1(): String = "m1 in B"
    def m3(): String
}

trait C {
    def m2(): String
    def m3(): String = "m3 in C"
}

class D extends B with C {
    def m2(x: Int): String = "m2 in D"
    def m4(): String = "m4 in D"
}
```

Write the result of each of the following calls, then comment on how overloading, overriding, and / or inheritance result in the given behavior. Assume the following has been run before each:
```scala
val d: D = new D()
val a: A = new A()
val a_from_d: A = d
```
1. `d.m1()`
2. `d.m2()`
3. `d.m2(46)`
4. `d.m3()`
5. `d.m4()`

6. `a_from_d.m1()`
7. `a_from_d.m2()`
8. `a.m1()`
9. `a.m2()`

Next, say whether each of the following casts is valid
1. `d.asInstanceOf[A]`
2. `a_from_d.asInstanceOf[D]`
3. `a_from_d.asInstanceOf[C]`
4. `a.asInstanceOf[D]`

### Solution
1. `"m1 in B"` - D doesn't implement m1 directly, so the method defined by B is used, which overrides m1 on A.
2. `"m2 in A"` - Method m2 on class D is overloaded, and this call is to the overload that also overrides m2 from B and implements m2 from C.
3. `"m2 in D"` - Method m2 on class D is overloaded, and this call is to the overload that is not inherited from either B or C
4. `"m3 in C"` - The implementation of m3 is inherited from C
5. `"m4 in D"` - m4 is defined only by D, and so doesn't override anything.
6. `"m1 in B"` - Though it has been cast to an A, the underlying object is still a D, and so we go through it's inheritance hierarchy and find that m1 is overriden in B.
7. `"m2 in A"` - Since nothing overrides m2, we use the only implementation in our hierarchy, found in A.
8. `"m1 in A"` - A does not inherit from anything, thus there is only one possible method to call.
9. `"m2 in A"` - A does not inherit from anything, thus there is only one possible method to call.


1. valid - D inherits from B which inherits from A, so (since inheritance is transitive) D "is an" A.
2. valid - The object is a D, it's just being treated as an A by the type system, so this is safe.
3. valid - The object is a D, it's just being treated as an A by the type system, so this is safe because D "is a" C (in other words, D <: C)
4. invalid - a is not a D, so it doesn't have the methods class D claims to provide.

In [1]:
class A {
    def m1(): String = "m1 in A"
    def m2(): String = "m2 in A"
}

abstract class B extends A {
    override def m1(): String = "m1 in B"
    def m3(): String
}

trait C {
    def m2(): String
    def m3(): String = "m3 in C"
}

class D extends B with C {
    def m2(x: Int): String = "m2 in D"
    def m4(): String = "m4 in D"
}

val d: D = new D()
val a: A = new A()
val a_from_d: A = d

// SOLUTIOMN

d.m1()
d.m2()
d.m2(46)
d.m3()
d.m4()

a_from_d.m1()
a_from_d.m2()
a.m1()
a.m2()

val d_casted1 = d.asInstanceOf[A]
val d_casted2 = a_from_d.asInstanceOf[D]
val d_casted3 = a_from_d.asInstanceOf[C]
// val d_casted4 = a.asInstanceOf[D]

defined class A
defined class B
defined trait C
defined class D
d: D = ammonite.$sess.cmd0$Helper$D@bc9c617
a: A = ammonite.$sess.cmd0$Helper$A@cd8e7b4
a_from_d: A = ammonite.$sess.cmd0$Helper$D@bc9c617
res0_7: String = "m1 in B"
res0_8: String = "m2 in A"
res0_9: String = "m2 in D"
res0_10: String = "m3 in C"
res0_11: String = "m4 in D"
res0_12: String = "m1 in B"
res0_13: String = "m2 in A"
res0_14: String = "m1 in A"
res0_15: String = "m2 in A"
d_casted1: A = ammonite.$sess.cmd0$Helper$D@bc9c617
d_casted2: D = ammonite.$sess.cmd0$Helper$D@bc9c617
d_casted3: C = ammonite.$sess.cmd0$Helper$D@bc9c617

## Exercise: Bringing it all together

We will create a simple class first and build to include more advanced topics each time.

### Initial attempt
The class below is our starting point

In [2]:
// Basic
import scala.collection.mutable.Queue

class Coke

class CokeVendingMachine {
    var money_collected: Int = 0

    def drop(): Coke = new Coke()
    
    def purchase(payment: Int): Coke = {
        this.money_collected += payment
        this.drop()
    }
}

import scala.collection.mutable.Queue


defined class Coke
defined class CokeVendingMachine

In [3]:
val coke_vm = new CokeVendingMachine()
coke_vm.purchase(2)

coke_vm: CokeVendingMachine = ammonite.$sess.cmd1$Helper$CokeVendingMachine@e357a67
res2_1: Coke = ammonite.$sess.cmd1$Helper$Coke@69133692

### Exercise: Inheritance and Generics
We want to have vending machines that dispense different items, refactor the code to use a base `VendingMachine` type with a type parameter representing the product.

In [4]:
// Inheritance + Generics
class Coke

abstract class VendingMachine[TProduct] {
    var money_collected: Int = 0

    def drop(): TProduct
    
    def purchase(payment: Int): TProduct = {
        this.money_collected += payment
        this.drop()
    }
}

class CokeVendingMachine extends VendingMachine[Coke] {
    def drop() = new Coke()
}

defined class Coke
defined class VendingMachine
defined class CokeVendingMachine

In [5]:
class Chips

class ChipVendingMachine extends VendingMachine[Chips] {
    def drop() = new Chips()
}

defined class Chips
defined class ChipVendingMachine

### Exercise: Multiple Inheritance
Next, we want to add 2 abilities to our vending machines:
1. Create a `Serviceable` trait or abstract class with a method `def perform_maintenance()` and implement it for our `VendingMachine` class (so that it empties `money_collected`).
2. Create a `Shakeable` trait or abstract class with a method `def shake(): ???` and implement it for our `VendingMachine` class (so that it empties calls drop). Figure out a way to replace the `???` with the product type.

In [6]:
// Traits
class Coke

trait Serviceable {
    def perform_maintenance()
}

trait Shakeable[TStuffThatFallsOut] {
    def shake(): TStuffThatFallsOut
}

abstract class VendingMachine[TProduct] extends Shakeable[TProduct] with Serviceable {
    var money_collected: Int = 0

    def drop(): TProduct
    
    def purchase(payment: Int): TProduct = {
        this.money_collected += payment
        this.drop()
    }
    
    // For Serviceable trait
    def perform_maintenance() {
        // Clear collected money
        this.money_collected = 0
    }
    
    // For Shakeable trait
    def shake(): TProduct = this.drop()
}

class CokeVendingMachine extends VendingMachine[Coke] {
    def drop(): Coke = new Coke()
}

defined class Coke
defined trait Serviceable
defined trait Shakeable
defined class VendingMachine
defined class CokeVendingMachine

In [7]:
val coke_vm = new CokeVendingMachine()
coke_vm.purchase(2)
assert(coke_vm.money_collected == 2)
coke_vm.perform_maintenance()
assert(coke_vm.money_collected == 0)
val coke: Coke = coke_vm.shake()

class Road extends Serviceable {
    var potholes = 20
    def perform_maintenance() {
        this.potholes -= 1
    }
}

coke_vm: CokeVendingMachine = ammonite.$sess.cmd5$Helper$CokeVendingMachine@1df8677f
res6_1: Coke = ammonite.$sess.cmd5$Helper$Coke@7fbfb2d6
coke: Coke = ammonite.$sess.cmd5$Helper$Coke@5a75d579
defined class Road

### Exercise: Type Constraints
Lets introduce vending machines that take specific payment types.
Use the following to change the `purchase` method to take a specific payment type as opposed to an integer using generics:
```scala
abstract class PaymentMethod {
    def get_amount(): Int = 5 // hardcoded for example
}
abstract class Card extends PaymentMethod
class Credit extends Card
class Cash extends PaymentMethod
```
Change `VendingMachine` as needed to specify a payment type.

In [8]:
// Type constraints
// Variance
trait Serviceable {
    def perform_maintenance()
}

class Coke

trait Shakeable[TStuffThatFallsOut] {
    def shake(): TStuffThatFallsOut
}

abstract class PaymentMethod {
    def get_amount(): Int = 5 // hardcoded for example
}
abstract class Card extends PaymentMethod
class Credit extends Card
class Cash extends PaymentMethod

abstract class VendingMachine[TPayment <: PaymentMethod, TProduct] extends Shakeable[TProduct] with Serviceable {
    var money_collected: Int = 0

    def drop(): TProduct
    
    def purchase(payment: TPayment): TProduct = {
        this.money_collected += payment.get_amount()
        this.drop()
    }
    
    // For Serviceable trait
    def perform_maintenance() {
        // Clear collected money
        this.money_collected = 0
    }
    
    // For Shakeable trait
    def shake(): TProduct = this.drop()
}

class CokeVendingMachine[TPayment <: PaymentMethod] extends VendingMachine[TPayment, Coke] {
    def drop(): Coke = new Coke()
}

defined trait Serviceable
defined class Coke
defined trait Shakeable
defined class PaymentMethod
defined class Card
defined class Credit
defined class Cash
defined class VendingMachine
defined class CokeVendingMachine

In [9]:
// This may not work under some versions of the scala kernel,
// but it's correct
val card_coke_vm = new CokeVendingMachine[Card]()

card_coke_vm: CokeVendingMachine[Card] = ammonite.$sess.cmd7$Helper$CokeVendingMachine@6feef8bc

## Inheritance + Generics = Variance

*Reminder*: `A <: B` means `A` is a subtype / inherits from / can replace `B`.

| Name          | Scala         | English |
|---------------|---------------|---------|
| Invariant     | `class C[T]`  | If `A <: B` then `C[A]` has no relation to `C[B]`. `T` can be used anywhere.  |
| Covariant     | `class C[+T]` | If `A <: B` then `C[A] <: C[B]` (same direction). `T` can only be used as input to methods.|
| Contravariant | `class C[-T]` | If `A <: B` then `C[A] >: C[B]`  (oposite direction). `T` can only be used as output of methods. |

### Exercise: Variance
For each of the following, mark whether the class should be in- co- or contra-variant with each type parameter:

1. A `RecyclingBin` class defined as follows:
    ```scala
    class RecyclingBin[TRecycleable] {
        def recycle(trash: TRecycleable): Unit
    }
    ```

2. If we made a class `ParkingMeter[PaymentMethod]` which takes in the specified type of payment, what would it's variance be?

3. If we made a class `MovieNight[Genre]` which plays a movie of the given genre, what would it's variance be?

4. If we made a class representing functions: `Function[InType, OutType]`, should it be co- or contravariant in `InType`? `OutType`?

### Solution

1. Contravariant. A recycling bin that takes all types of recycleables (paper, plastic, glass) can stand in for a paper recycling bin, but people can't put plastic in a paper recycling bin.
2. Contravariant. A parking meter that takes any coin can stand in for one that only takes quarters, but the other way will not work.
3. Covariant. A `MovieNight[FastAndFurious]` could stand in for a `MovieNight[Fiction]`, but playing an arbitrary fiction movie on Fast and Furious night will absolutely not suffice.
4. Contravariant with `InType`, covariant with `OutType`. Since `Int <: Complex` and `PositiveInt <: Real`, a `Complex => PositiveInt` can stand in for a `Int => Real`. If we try it the other way then `Int => Real` can't take in `3 + 4i` and it won't necessarily produce a positive integer.

### Exercise: Back to the vending machines with variance
Finally, we want the ability to use a vending machine that takes any payment type and dispenses coke in a place where a vending machine that takes cash and produces soda is expected. In other words, we want to define the variance of `VendingMachine`.

Add the proper variance to any type parameters in order to make this happen.
Use the following inheritance structure for coke:
```scala
abstract class Soda
class Coke extends Soda
```

In [10]:
// Variance
import scala.collection.mutable.Queue

trait Serviceable {
    def perform_maintenance()
}

abstract class PaymentMethod {
    def get_amount(): Int = 5 // hardcoded for example, $5
}
abstract class Card extends PaymentMethod
class Credit extends Card
class Cash extends PaymentMethod

abstract class Soda
class Coke extends Soda

trait Shakeable[+TStuffThatFallsOut] {
    def shake(): TStuffThatFallsOut
}

abstract class VendingMachine[-TPayment <: PaymentMethod, +TProduct] extends Shakeable[TProduct] with Serviceable {
    var money_collected: Int = 0

    def drop(): TProduct
    
    def purchase(payment: TPayment): TProduct = {
        this.money_collected += payment.get_amount()
        this.drop()
    }
    
    // For Serviceable trait
    def perform_maintenance() {
        // Clear collected money
        this.money_collected = 0
    }
    
    // For Shakeable trait
    def shake(): TProduct = this.drop()
}

class CokeVendingMachine[-TPayment <: PaymentMethod] extends VendingMachine[TPayment, Coke] {
    def drop(): Coke = new Coke()
}

import scala.collection.mutable.Queue


defined trait Serviceable
defined class PaymentMethod
defined class Card
defined class Credit
defined class Cash
defined class Soda
defined class Coke
defined trait Shakeable
defined class VendingMachine
defined class CokeVendingMachine

In [11]:
// This may not work under some versions of the scala kernel,
// but it's correct
val card_coke_vm = new CokeVendingMachine[Card]()
val credit_soda_vm: VendingMachine[Credit, Soda] = card_coke_vm

card_coke_vm: CokeVendingMachine[Card] = ammonite.$sess.cmd9$Helper$CokeVendingMachine@6cd50cf3
credit_soda_vm: VendingMachine[Credit, Soda] = ammonite.$sess.cmd9$Helper$CokeVendingMachine@6cd50cf3